<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=bedf8000a93243af9ee244c938a0d1a0ca45e4a948576eef0fbd9cf29c25645a
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-24 14:30:35
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  66.76 K
Current:  1.42 C
-------------------
Today PnL: -12.42 K (-0.09%)
Current PnL: -25.41 L (-16.41%)
CY Booked + Current PnL: -11.12 L (-7.18%)
-------------------
Total profit:  1.61 L
Total loss:  -27.02 L
-------------------
Total Booked + Current PnL: 15.86 L (12.41%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.09%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.32 C
Est FTT PnL: 90.04 L (63.57%)
Deployed:  1.28 C
Current:  1.42 C
CAGR/XIRR %: 5.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.09,-19.20,23.87,0.09,19437.0,-19347.0,81430.0,88.90,38.0,M-SC,2.63,253.0,-1.00,0.58,6.98,OX40N,NTT,DURABLES
82,VOLTAS,1530.00,0.74,9.20,9.61,19.69,20122.0,17643.0,209385.0,-1.49,56.0,X-MC,2.48,78.0,0.88,1.49,16.62,XY25,NTT,AC
50,MASFIN,398.61,-0.71,-1.81,24.29,22.05,23369.0,-1770.0,96210.0,-15.54,55.0,H-SC,6.62,164.0,-0.08,0.68,39.85,XR,ATH,FINANCE
52,NESTLEIND,1377.00,0.43,11.25,9.12,21.40,26930.0,29859.0,295285.0,3.28,58.0,X-LC,5.53,12.0,1.11,2.09,18.64,XY25,NTT,FMCG
43,ITC,452.00,-0.27,-0.43,11.26,10.78,26943.0,-1037.0,239281.0,-40.74,55.0,X-LC,2.03,5.0,-0.04,1.70,4.78,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,8.66,-34.67,72.81,12.90,103936.0,-75746.0,142750.0,110.34,64.0,H-SC,12.55,161.0,-0.73,1.01,18.70,OX40N,NTT,FINANCE
6,ASIANTILES,137.00,4.91,2.04,78.04,81.67,74705.0,1916.0,95726.0,8622.22,77.0,L-SC,13.20,271.0,0.03,0.68,85.38,XR,NTT,CERAMICS
80,VALIANTORG,838.00,3.53,-41.84,199.45,74.15,189775.0,-68456.0,95149.0,-270.93,56.0,H-SC,22.11,149.0,-0.36,0.67,21.97,XR,NTT,CHEMICALS
26,GILLETTE,11206.78,3.07,-2.34,35.59,32.42,88245.0,-5946.0,247950.0,-19.73,49.0,X-SC,10.70,85.0,-0.07,1.76,12.40,X40,ATH,FMCG
47,KANSAINER,340.00,2.90,-22.99,47.34,13.47,98318.0,-61983.0,207684.0,-68.31,52.0,H-SC,6.66,158.0,-0.63,1.47,5.60,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,GREENPANEL,537.0,-2.72,-37.81,130.18,43.14,148210.0,-69228.0,113850.0,155.17,40.0,M-SC,10.72,240.0,-0.47,0.81,7.09,XY24,NTT,MISC
57,RAJOOENG,143.1,-2.37,-39.36,130.21,39.61,80939.0,-40340.0,62160.0,-58.91,30.0,H-SC,31.13,135.0,-0.50,0.44,0.00,AR,ATH,MISC
10,BANDHANBNK,400.0,-1.95,-29.56,173.17,92.42,339787.0,-82343.0,196216.0,37.93,46.0,H-SC,9.42,170.0,-0.24,1.39,13.40,XY24,NTT,BANKS
83,WHIRLPOOL,2270.0,-1.84,-25.87,145.41,81.92,123744.0,-29698.0,85100.0,-56.60,30.0,M-SC,21.79,236.0,-0.24,0.60,1.51,XR,NTT,DURABLES
37,IEX,219.0,-1.70,-5.23,56.83,48.63,107688.0,-10463.0,189491.0,-36.95,45.0,H-SC,13.42,136.0,-0.10,1.34,5.72,XR,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.34,-1.84,111.80,107.91,162036.0,-2710.0,144934.0,-20.15,55.0,M-SC,11.82,216.0,-0.02,1.03,8.07,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.10,-17.86,21.77,0.02,31214.0,-31178.0,143381.0,118.75,38.0,M-SC,11.33,234.0,-1.00,1.02,24.72,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.09,-19.20,23.87,0.09,19437.0,-19347.0,81430.0,88.90,38.0,M-SC,2.63,253.0,-1.00,0.58,6.98,OX40N,NTT,DURABLES
72,TATAELXSI,9161.00,-0.84,-22.41,70.17,32.03,71774.0,-29549.0,102286.0,-16.04,59.0,H-SC,8.95,148.0,-0.41,0.73,14.57,OX40N,NTT,IT
47,KANSAINER,340.00,2.90,-22.99,47.34,13.47,98318.0,-61983.0,207684.0,-68.31,52.0,H-SC,6.66,158.0,-0.63,1.47,5.60,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,4.91,2.04,78.04,81.67,74705.0,1916.0,95726.0,8622.22,77.0,L-SC,13.20,271.0,0.03,0.68,85.38,XR,NTT,CERAMICS
84,WIPRO,420.0,-1.31,9.86,56.80,72.27,105737.0,16708.0,186156.0,-5.63,66.0,M-LC,3.88,99.0,0.16,1.32,16.64,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.31,9.86,56.80,72.27,105737.0,16708.0,186156.0,-5.63,66.0,M-LC,3.88,99.0,0.16,1.32,16.64,XR,NTT,IT
6,ASIANTILES,137.00,4.91,2.04,78.04,81.67,74705.0,1916.0,95726.0,8622.22,77.0,L-SC,13.20,271.0,0.03,0.68,85.38,XR,NTT,CERAMICS
25,FINCABLES,1641.55,0.34,-1.84,111.80,107.91,162036.0,-2710.0,144934.0,-20.15,55.0,M-SC,11.82,216.0,-0.02,1.03,8.07,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-1.10,-3.96,115.24,106.72,136508.0,-4881.0,118455.0,-52.54,41.0,H-SC,4.50,139.0,-0.04,0.84,18.50,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-1.07,-3.21,40.13,35.63,81402.0,-6729.0,202845.0,-16.74,42.0,H-MC,2.37,119.0,-0.08,1.44,12.93,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.56,-18.36,90.43,55.47,179617.0,-44660.0,198626.0,-27.60,30.0,X-MC,15.62,64.0,-0.25,1.41,1.77,X40N,ATH,FINANCE
8,AWL,485.00,-1.00,-25.01,104.68,53.50,236877.0,-75464.0,226287.0,-63.76,31.0,X-MC,5.93,58.0,-0.32,1.61,0.51,XY24,NTT,FMCG
34,HONAUT,58357.33,-0.37,-18.39,75.62,43.32,100514.0,-29952.0,132920.0,-29.91,31.0,X-SC,5.98,90.0,-0.30,0.94,2.06,X40N,ATH,ELECTRICAL
20,DABUR,735.00,-0.60,-4.16,49.94,43.70,118731.0,-10321.0,237747.0,-11.99,32.0,X-MC,2.11,72.0,-0.09,1.69,10.26,XY24,BTT,FMCG
3,ACC,3906.00,-0.95,-26.98,124.81,64.15,216857.0,-64201.0,173750.0,-57.19,33.0,X-SC,4.22,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BERGEPAINT,680.0,1.69,-1.44,22.91,21.14,51334.0,-3273.0,224066.0,-8.42,54.0,X-MC,1.06,74.0,-0.06,1.59,26.30,XY24,NTT,PAINTS
15,CAMS,950.0,0.22,-0.17,24.69,24.48,62077.0,-429.0,251427.0,-80.87,49.0,X-SC,1.34,86.0,-0.01,1.78,22.93,X40N,NTT,MISC
78,UNITDSPR,1644.0,-1.23,4.61,15.45,20.77,37841.0,10784.0,244928.0,-4.44,49.0,X-MC,1.95,62.0,0.28,1.74,11.84,X40N,NTT,BREWERIES
43,ITC,452.0,-0.27,-0.43,11.26,10.78,26943.0,-1037.0,239281.0,-40.74,55.0,X-LC,2.03,5.0,-0.04,1.70,4.78,X40,NTT,FMCG
20,DABUR,735.0,-0.60,-4.16,49.94,43.70,118731.0,-10321.0,237747.0,-11.99,32.0,X-MC,2.11,72.0,-0.09,1.69,10.26,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.95,-26.98,124.81,64.15,216857.0,-64201.0,173750.0,-57.19,33.0,X-SC,4.22,82.0,-0.30,1.23,0.00,XY24,BTT,CEMENT
19,COLPAL,3726.84,-0.57,-20.49,77.97,41.51,163277.0,-53955.0,209410.0,-5.20,37.0,X-MC,7.89,61.0,-0.33,1.49,0.32,XY25,ATH,FMCG
8,AWL,485.00,-1.00,-25.01,104.68,53.50,236877.0,-75464.0,226287.0,-63.76,31.0,X-MC,5.93,58.0,-0.32,1.61,0.51,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.10,-38.75,120.76,35.20,95164.0,-49866.0,78804.0,3.25,35.0,X-SC,17.01,92.0,-0.52,0.56,1.08,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-0.56,-18.36,90.43,55.47,179617.0,-44660.0,198626.0,-27.60,30.0,X-MC,15.62,64.0,-0.25,1.41,1.77,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,2.36,-26.56,94.50,42.84,45116.0,-17264.0,47742.0,-52.07,56.0,X-SC,33.75,83.0,-0.38,0.34,8.01,XY24,NTT,MISC
59,RELAXO,1176.00,0.30,-47.59,190.62,52.31,145012.0,-69086.0,76074.0,-44.25,44.0,X-SC,7.35,91.0,-0.48,0.54,1.83,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.10,-38.75,120.76,35.20,95164.0,-49866.0,78804.0,3.25,35.0,X-SC,17.01,92.0,-0.52,0.56,1.08,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.25,-0.55,24.77,24.08,31271.0,-704.0,126246.0,-8.54,49.0,X-SC,6.39,89.0,-0.02,0.90,18.95,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.37,-18.39,75.62,43.32,100514.0,-29952.0,132920.0,-29.91,31.0,X-SC,5.98,90.0,-0.30,0.94,2.06,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.18,-8.82,32.39,20.71,111701.0,-33373.0,344864.0,-21.36,70.0,X-LC,3.20,1.0,-0.30,2.45,15.23,X40,ATH,IT
41,INFY,1972.00,-0.41,13.10,18.70,34.24,66175.0,40990.0,353878.0,-12.06,67.0,X-LC,6.15,2.0,0.62,2.51,22.47,X40,NTT,IT
75,TMPV,600.00,1.14,-25.55,65.18,22.98,109623.0,-57713.0,168185.0,-23.08,54.0,X-LC,6.41,3.0,-0.53,1.19,5.78,XY24,NTT,AUTO
81,VBL,671.64,-0.83,-3.49,40.58,35.67,123689.0,-11038.0,304804.0,-14.99,54.0,X-LC,3.46,4.0,-0.09,2.16,9.88,X40N,ATH,FMCG
43,ITC,452.00,-0.27,-0.43,11.26,10.78,26943.0,-1037.0,239281.0,-40.74,55.0,X-LC,2.03,5.0,-0.04,1.70,4.78,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.32,-37.45,112.46,32.89,54307.0,-28916.0,48290.0,-696.43,53.0,L-MC,6.79,259.0,-0.53,0.34,32.99,XR,NTT,BANKS
56,RAJESHEXPO,518.00,-1.66,-57.63,136.20,0.07,79155.0,-79060.0,58117.0,2030.05,67.0,L-SC,3.99,268.0,-1.00,0.41,44.54,OX40N,NTT,JEWELLERY
50,MASFIN,398.61,-0.71,-1.81,24.29,22.05,23369.0,-1770.0,96210.0,-15.54,55.0,H-SC,6.62,164.0,-0.08,0.68,39.85,XR,ATH,FINANCE
6,ASIANTILES,137.00,4.91,2.04,78.04,81.67,74705.0,1916.0,95726.0,8622.22,77.0,L-SC,13.20,271.0,0.03,0.68,85.38,XR,NTT,CERAMICS
13,BSOFT,831.70,-1.44,-18.54,85.11,50.79,97512.0,-26081.0,114572.0,4.37,68.0,H-SC,8.41,151.0,-0.27,0.81,33.40,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,4.91,2.04,78.04,81.67,74705.0,1916.0,95726.0,8622.22,77.0,L-SC,13.20,271.0,0.03,0.68,85.38,XR,NTT,CERAMICS
13,BSOFT,831.70,-1.44,-18.54,85.11,50.79,97512.0,-26081.0,114572.0,4.37,68.0,H-SC,8.41,151.0,-0.27,0.81,33.40,XR,ATH,IT
56,RAJESHEXPO,518.00,-1.66,-57.63,136.20,0.07,79155.0,-79060.0,58117.0,2030.05,67.0,L-SC,3.99,268.0,-1.00,0.41,44.54,OX40N,NTT,JEWELLERY
61,ROUTE,2227.21,-1.38,-46.49,215.89,69.05,153735.0,-61856.0,71210.0,-56.81,52.0,H-SC,20.65,140.0,-0.40,0.51,9.82,SR,ATH,IT
80,VALIANTORG,838.00,3.53,-41.84,199.45,74.15,189775.0,-68456.0,95149.0,-270.93,56.0,H-SC,22.11,149.0,-0.36,0.67,21.97,XR,NTT,CHEMICALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.34
1,25,44.77
2,50,76.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.02
MC    30.73
LC    24.24
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.76
X40      23.05
X40N     12.90
XR        9.92
XY25      9.13
AR        8.89
OX40N     7.59
SR        1.00
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.21
H-SC    23.85
X-LC    20.75
M-SC    11.55
X-SC     8.05
H-MC     4.80
L-SC     1.57
M-MC     1.38
M-LC     1.32
H-LC     1.11
L-LC     1.06
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.21,-7.62,42.22
IT,13.12,-13.73,70.92
FINANCE,9.58,-17.84,69.09
MISC,7.24,-27.19,79.22
ELECTRICAL,5.96,-11.69,52.61
PAINTS,5.82,-12.55,29.27
INSURANCE,4.73,-1.63,36.41
PHARMA,4.04,-3.25,35.74
AUTO,2.86,-29.75,74.74


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3148152.0,21
AR,1319324.0,10
XR,1260621.0,13
X40,973737.0,14
X40N,866989.0,9
OX40N,722628.0,10
XY25,354890.0,6
SR,280019.0,2
MH,78089.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3614048.0,25
M-SC,1429480.0,15
X-MC,1416557.0,16
X-LC,800021.0,11
X-SC,784256.0,8
H-MC,402241.0,3
L-SC,239493.0,3
M-LC,105737.0,1
H-LC,77526.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1262428.0      6
           AR         906014.0      5
M-SC       XY24       807796.0      6
H-SC       XR         764727.0      7
X-MC       X40        487349.0      7
           XY24       406942.0      3
           X40N       338867.0      4
H-SC       OX40N      322771.0      4
M-SC       OX40N      320702.0      5
X-SC       X40N       307603.0      3
X-LC       X40        302979.0      5
X-SC       XY24       293244.0      3
H-SC       SR         280019.0      2
X-LC       X40N       220519.0      2
H-MC       AR         210314.0      2
           XY24       191927.0      1
X-LC       XY24       185815.0      2
X-SC       X40        183409.0      2
X-MC       XY25       183399.0      2
M-SC       XR         175512.0      2
L-SC       XR         160338.0      2
M-SC       AR         125470.0      2
M-LC       XR         105737.0      1
X-LC       XY25        90708.0      2
L-SC       OX40N       79155.0      1
H-SC       MH          78089.0      1
H-LC       AR          77526.0      1
L-MC       XR          54307.0      1
M-MC       XY25        50291.0      1
L-LC       XY25        30492.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
